In [28]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from IPython.display import Image

import glob
import re
import os
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import statistics
import collections
import itertools
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import gc
import time
import math
from scipy import signal
from scipy.fftpack import fft, fftfreq
from scipy.stats import iqr
from scipy.stats import pearsonr
from scipy.stats import skew
from scipy.stats import kurtosis
from spectrum import *
from statsmodels.robust import mad as median_deviation
from sklearn.preprocessing import minmax_scale
import os

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.manifold import TSNE
from lightgbm import LGBMClassifier
from collections import Counter

In [29]:
# 混同行列
def generate_cm(df, index, columns, filename, index_name="True Label", columns_name="Estimated Label", figure_size=(10, 8), fontsize=20,  font="Times New Roman", bottom=None, top=None, right=None, left=None):
    cm2 = pd.DataFrame(df, index=index, columns=columns)
    cm3 = cm2.astype('float').div(cm2.sum(axis=1), axis="index")
    cm3_object = pd.DataFrame(cm3.values, index=index, columns=columns, dtype=str)
    
    for i in range(cm3_object.shape[0]):
        for j in range(cm3_object.shape[1]):
            cm3_object.iat[i, j] = "{0:.1f}".format(cm3.iat[i, j]) + "\n" + "(" + str(cm2.iat[i, j]) + ")"
    
    cm3.index.name = "True label"
    cm3.columns.name = "Estimated label"
    sns.set()
    sns.set('talk', 'whitegrid', 'dark', font_scale=1)

    plt.figure()
    plt.rcParams["font.size"] = fontsize
    plt.rcParams['font.family'] = font
    plt.figure(figsize=figure_size)
    plt.xlabel('Predict')
    plt.ylabel('Actual')

#     plt.subplots_adjust(bottom=bottom, top=top, left=left, right=right)
    plt.tight_layout()
    #     plt.subplots_adjust()

    sns.heatmap(cm3, annot=cm3_object.values, fmt="", linewidth=2, cmap="gray_r", linecolor="black", vmax=1, vmin=0)


    plt.savefig(filename, bbox_inches="tight", pad_inches=0.05)
    plt.close("all")

In [23]:
def run_closs_val(terget_df, terget_dir, terget_class, condition, classifiers, features_list, val_type):
    
    
    X = terget_df[ features_list].values
    Y = terget_df.label.values
    #groups = terget_df.case.values

    labels = np.unique(Y)
    
    for i in range(len(classifiers)):
        name, clf = classifiers[i]
        print(name)
        scaler = StandardScaler()
        #pca = PCA(n_components=0.95)
        pipeline = make_pipeline(scaler, clf)
        #cv = LeaveOneGroupOut()
        #cv = StratifiedKFold(10)
        if (val_type == "person-out"):
            print(val_type)
            cv = LeaveOneGroupOut()
        else:
            print("10 fold")
            cv = StratifiedKFold(10)


        
        y_pred = cross_val_predict(pipeline, X, Y, #groups=groups, 
                                   cv=cv, n_jobs=7, verbose=1)

        cr_dict = classification_report(Y, y_pred, output_dict=True)
        cr_df = pd.DataFrame(cr_dict).T
        cr_df.to_csv(terget_dir + "/"+condition+ '/clf_reports' + '/'+condition + '_'+ terget_class + '_'+'result_report_'+name+'.csv')

        cm = confusion_matrix(Y, y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'result_cm_' + name + ".png"

        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)
        

#         ddf = pd.concat([pd.Series(y_pred), pd.Series(Y), pd.Series(groups)] ,axis = 1)
#         ddf.columns=["y_pred","y", "session"]
        
        
        ddf = pd.concat([pd.Series(y_pred), pd.Series(Y)] ,axis = 1)
        ddf.columns=["y_pred","y"]
        
        ddf.to_csv(terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'y_pred_true_user_' + name +'.csv')
        
        print(name,"fin")
    #     Image(cm_file_name)

In [43]:
def run_closs_val_PI(terget_df, terget_dir, terget_class, condition, classifiers, features_list):
    
    
    X = terget_df[ features_list].values
    Y = terget_df.label.values
        
    groups = terget_df.user.values
    labels = np.unique(Y)
    #val_type = condition.split("_")[-1]
    
    
    for i in range(len(classifiers)):
        name, clf = classifiers[i]
        print(name)
        scaler = StandardScaler()
        #pca = PCA(n_components=0.95)
        pipeline = make_pipeline(scaler, clf)
        
        
        cv = LeaveOneGroupOut()
#         if (val_type == "person-out"):
#             print(val_type)
#             cv = LeaveOneGroupOut()
#         else:
#             print("10 fold")
#             cv = StratifiedKFold(10)

        # cv = StratifiedKFold(5)
        y_pred = cross_val_predict(pipeline, X, Y,
                                   groups=groups, cv=cv,
                                   n_jobs=7, verbose=1)

        cr_dict = classification_report(Y, y_pred, output_dict=True)
        cr_df = pd.DataFrame(cr_dict).T
        cr_df.to_csv(terget_dir + "/"+condition+ '/clf_reports' + '/'+condition + '_'+ terget_class + '_'+'result_report_'+name+'.csv')

        cm = confusion_matrix(Y, y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'result_cm_' + name + ".png"

        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)
        

        ddf = pd.concat([pd.Series(y_pred), pd.Series(Y), pd.Series(groups)] ,axis = 1)
        ddf.columns=["y_pred","y","user"]
        ddf.to_csv(terget_dir+ "/"+condition+ '/'+condition + '_'+ terget_class + '_'+ 'y_pred_true_user_' + name +'.csv')
        
        print(name,"fin")
    #     Image(cm_file_name)

In [94]:
def run_closs_val_PD(target_df, target_dir, target_class, condition, classifiers, features_list):
    
    for i in range(len(classifiers)):
        name, clf = classifiers[i]
        print(name)
        scaler = StandardScaler()
        #pca = PCA(n_components=0.95)
        pipeline = make_pipeline(scaler, clf)

        print("10 fold")
        cv = StratifiedKFold(10)
        
        
        users = np.unique(target_df.user)
        df_u_all = pd.DataFrame()
        for j , u_name in enumerate(users):
            tmp_df = target_df[target_df.user == u_name]
            print(u_name)

            X = tmp_df[features_list].values
            Y = tmp_df.label.values
#             if(target_class == "action"):
#                 print("action")
#                 Y = tmp_df.action.values
#             elif(target_class == "place"):
#                 print("place")
#                 Y = tmp_df.place.values
#             else:
#                 return 0

            groups = tmp_df.user.values
            labels = np.unique(Y)
            #val_type = condition .split("_")[-1]

            # cv = StratifiedKFold(5)
            y_pred = cross_val_predict(pipeline, X, Y,
                                       groups=groups, cv=cv,
                                       n_jobs=7, verbose=1)
            
            u_name = str(u_name)

            cr_dict = classification_report(Y, y_pred, output_dict=True)
            cr_df = pd.DataFrame(cr_dict).T
            cr_df.to_csv(target_dir + "/"+condition+  '/clf_reports' + '/'+ u_name + '_'+condition + '_'+ target_class + '_'+'result_report_'+name+'.csv')

            cm = confusion_matrix(Y, y_pred, labels=labels)
            cm_df = pd.DataFrame(cm, index=labels, columns=labels)
            cm_file_name =  target_dir + "/"+condition+  "/" + u_name + '_'+condition + '_'+ target_class + '_'+ 'result_cm_' + name + ".png"

            generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)

            ddf = pd.concat([pd.Series(y_pred), pd.Series(Y), pd.Series(groups)] ,axis = 1)
            ddf.columns=["y_pred","y","user"]
            ddf.to_csv(target_dir + "/"+condition+  "/" + u_name +'_'+condition + '_'+ target_class + '_'+ 'y_pred_true_user_' + name +'.csv')

            df_u_all = pd.concat([df_u_all, ddf])

            print(u_name, name,"fin")

        df_u_all.to_csv(target_dir + "/"+condition+  "/" + condition + '_'+ target_class + '_'+ 'y_pred_true_user_' + name +'.csv')

        cr_dict = classification_report(df_u_all.y, df_u_all.y_pred, output_dict=True)
        cr_df = pd.DataFrame(cr_dict).T

        cr_df.to_csv(target_dir+ "/"+condition+ '/clf_reports' + '/'+ condition + '_'+ target_class +'result_report_'+name+'.csv')

        cm = confusion_matrix(df_u_all.y, df_u_all.y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  target_dir+ "/"+condition+ '/'+ condition + '_'+ target_class +  'result_cm_' + name + ".png"
        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)

            #     Image(cm_file_name)

In [95]:
# 特徴量重要度を棒グラフでプロットする関数 
def plot_feature_importance(df): 
    n_features = len(df)                              # 特徴量数(説明変数の個数) 
    df_plot = df.sort_values('importance')            # df_importanceをプロット用に特徴量重要度を昇順ソート 
    f_importance_plot = df_plot['importance'].values  # 特徴量重要度の取得 
    plt.barh(range(n_features), f_importance_plot, align='center') 
    cols_plot = df_plot['feature'].values             # 特徴量の取得 
    plt.yticks(np.arange(n_features), cols_plot)      # x軸,y軸の値の設定
    plt.xlabel('Feature importance')                  # x軸のタイトル
    plt.ylabel('Feature')                             # y軸のタイトル

In [25]:
all_feature145_names = ['t-Mean-1',
 't-Mean-2',
 't-Mean-3',
 't-STD-1',
 't-STD-2',
 't-STD-3',
 't-Mad-1',
 't-Mad-2',
 't-Mad-3',
 't-Max-1',
 't-Max-2',
 't-Max-3',
 't-Min-1',
 't-Min-2',
 't-Min-3',
 't-SMA-1',
 't-Energy-1',
 't-Energy-2',
 't-Energy-3',
 't-IQR-1',
 't-IQR-2',
 't-IQR-3',
 't-ropy-1',
 't-ropy-2',
 't-ropy-3',
 't-ARCoeff-1',
 't-ARCoeff-2',
 't-ARCoeff-3',
 't-ARCoeff-4',
 't-ARCoeff-5',
 't-ARCoeff-6',
 't-ARCoeff-7',
 't-ARCoeff-8',
 't-ARCoeff-9',
 't-ARCoeff-10',
 't-ARCoeff-11',
 't-ARCoeff-12',
 't-Correlation-1',
 't-Correlation-2',
 't-Correlation-3',
 't-Mean-mag',
 't-STD-mag',
 't-Mad-mag',
 't-Max-mag',
 't-Min-mag',
 't-SMA-mag',
 't-Energy-mag',
 't-IQR-mag',
 't-ropy-mag',
 't-ARCoeff-mag1',
 't-ARCoeff-mag2',
 't-ARCoeff-mag3',
 't-ARCoeff-mag4',
 'f-Mean-1',
 'f-Mean-2',
 'f-Mean-3',
 'f-STD-1',
 'f-STD-2',
 'f-STD-3',
 'f-Mad-1',
 'f-Mad-2',
 'f-Mad-3',
 'f-Max-1',
 'f-Max-2',
 'f-Max-3',
 'f-Min-1',
 'f-Min-2',
 'f-Min-3',
 'f-SMA-1',
 'f-Energy-1',
 'f-Energy-2',
 'f-Energy-3',
 'f-IQR-1',
 'f-IQR-2',
 'f-IQR-3',
 'f-ropy-1',
 'f-ropy-2',
 'f-ropy-3',
 'f-MaxInds-1',
 'f-MaxInds-2',
 'f-MaxInds-3',
 'f-MeanFreq-1',
 'f-MeanFreq-2',
 'f-MeanFreq-3',
 'f-Skewness-1',
 'f-Skewness-2',
 'f-Skewness-3',
 'f-Kurtosis-1',
 'f-Kurtosis-2',
 'f-Kurtosis-3',
 'f-BandsEnergyOld-1',
 'f-BandsEnergyOld-2',
 'f-BandsEnergyOld-3',
 'f-BandsEnergyOld-4',
 'f-BandsEnergyOld-5',
 'f-BandsEnergyOld-6',
 'f-BandsEnergyOld-7',
 'f-BandsEnergyOld-8',
 'f-BandsEnergyOld-9',
 'f-BandsEnergyOld-10',
 'f-BandsEnergyOld-11',
 'f-BandsEnergyOld-12',
 'f-BandsEnergyOld-13',
 'f-BandsEnergyOld-14',
 'f-BandsEnergyOld-15',
 'f-BandsEnergyOld-16',
 'f-BandsEnergyOld-17',
 'f-BandsEnergyOld-18',
 'f-BandsEnergyOld-19',
 'f-BandsEnergyOld-20',
 'f-BandsEnergyOld-21',
 'f-BandsEnergyOld-22',
 'f-BandsEnergyOld-23',
 'f-BandsEnergyOld-24',
 'f-BandsEnergyOld-25',
 'f-BandsEnergyOld-26',
 'f-BandsEnergyOld-27',
 'f-BandsEnergyOld-28',
 'f-BandsEnergyOld-29',
 'f-BandsEnergyOld-30',
 'f-BandsEnergyOld-31',
 'f-BandsEnergyOld-32',
 'f-BandsEnergyOld-33',
 'f-BandsEnergyOld-34',
 'f-BandsEnergyOld-35',
 'f-BandsEnergyOld-36',
 'f-BandsEnergyOld-37',
 'f-BandsEnergyOld-38',
 'f-BandsEnergyOld-39',
 'f-BandsEnergyOld-40',
 'f-BandsEnergyOld-41',
 'f-BandsEnergyOld-42',
 'f-Mean-mag',
 'f-STD-mag',
 'f-Mad-mag',
 'f-Max-mag',
 'f-Min-mag',
 'f-SMA-mag',
 'f-Energy-mag',
 'f-IQR-mag',
 'f-ropy-mag',
 'f-MaxInds-mag',
 'f-MeanFreq-mag',
 'f-Skewness-mag',
 'f-Kurtosis-mag']



def add_acc(n):
    return "acc_" + n

acc_feature145_names = list(map(add_acc, all_feature145_names))

In [6]:
classifiers = [
               ("RF", RandomForestClassifier(random_state=0)),
               ("ANN", MLPClassifier()),
               ("LR", LogisticRegression()),
               ("DT", DecisionTreeClassifier()),
               ("ET", ExtraTreeClassifier()),
               ("LR", LogisticRegression()),
               ("NB", GaussianNB()),
               ("SVM", SVC()),
               ("KNN", KNeighborsClassifier()),
               ("LightGBM", LGBMClassifier())]

/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Bottom_2021-07-16T15.18.45.753_E2EE3F7D14CE_Accelerometer.csv
/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Middle_2021-07-16T15.18.10.660_CEF5C0D2C881_Accelerometer.csv
/Users/y-nakamura/yugo_workspace/bar_exercise/data/raw/20210716/Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer.csv

In [7]:
os.getcwd()


'/Users/y-nakamura/yugo_workspace/bar_exercise/notebook'

In [66]:

terger_dir ="/Users/y-nakamura/yugo_workspace/bar_exercise/data/features/2021_dpsws"

os.chdir(terger_dir)

# df_bottom = pd.read_csv(terger_dir + "/Bottom_2021-07-16T15.18.45.753_E2EE3F7D14CE_Accelerometer_features.csv")
# df_middle = pd.read_csv(terger_dir + "/Middle_2021-07-16T15.18.10.660_CEF5C0D2C881_Accelerometer_features.csv")
# df_upper = pd.read_csv(terger_dir + "/Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer_features.csv")
#df_upper = pd.read_csv(terger_dir + "/0721_Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer - Upper_2021-07-16T15.17.48.292_D738727472C3_Accelerometer_features.csv")


df_upper = pd.read_csv(terger_dir + "/0721_Upper_Accelerometer_features128.csv")

df_middle = pd.read_csv(terger_dir + "/0721_Middle_Accelerometer_features128.csv")

df_bottom = pd.read_csv(terger_dir + "/0721_Bottom_Accelerometer_features128.csv")

#df_upper = pd.read_csv(terger_dir + "/0721_Accelerometer_Upper_Accelerometer_features64.csv")


df_upper.head()

,acc_t-Mean-1,acc_t-Mean-2,acc_t-Mean-3,acc_t-STD-1,acc_t-STD-2,acc_t-STD-3,acc_t-Mad-1,acc_t-Mad-2,acc_t-Mad-3,acc_t-Max-1,...,acc_f-SMA-mag,acc_f-Energy-mag,acc_f-IQR-mag,acc_f-ropy-mag,acc_f-MaxInds-mag,acc_f-MeanFreq-mag,acc_f-Skewness-mag,acc_f-Kurtosis-mag,label,user
0,0.012326,-0.980078,-0.221543,0.000586,0.001016,0.000863,0.000000,0.000000,0.001483,0.013,...,0.031543,0.063112,0.000133,-1.912936,0.0,0.070118,7.811263,59.015843,0,0
1,0.012333,-0.980031,-0.221388,0.000614,0.000797,0.000901,0.000000,0.000000,0.001483,0.013,...,0.031509,0.063102,0.000094,-1.934977,0.0,0.058999,7.811264,59.015855,0,0
2,0.012264,-0.980093,-0.221388,0.000536,0.000802,0.000847,0.000000,0.000000,0.001483,0.013,...,0.031510,0.063109,0.000100,-1.936172,0.0,0.055842,7.811264,59.015853,0,0
3,0.012380,-0.980256,-0.221481,0.000546,0.000800,0.000973,0.000000,0.001483,0.001483,0.014,...,0.031508,0.063132,0.000068,-1.942563,0.0,0.051160,7.811264,59.015854,0,0
4,0.012473,-0.980163,-0.221690,0.000585,0.000713,0.001048,0.001483,0.000000,0.001483,0.014,...,0.031501,0.063126,0.000083,-1.946101,0.0,0.052189,7.811265,59.015861,0,0


In [68]:
np.unique(df_upper.user.values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [70]:
np.unique(df_middle.user.values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [72]:
np.unique(df_bottom.user.values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [74]:
df_upper.shape

(1063, 147)

In [75]:
df_middle.shape

(1594, 147)

In [76]:
df_bottom.shape

(2529, 147)

In [13]:
user_list = [1, 2, 3, 4, 5, 6, 7, 8]

In [14]:
df = df_upper[df_upper.user.isin(user_list)]

In [78]:
df.shape

(652, 147)

In [80]:
df_middle[df_middle.user.isin(user_list)].shape

(652, 147)

In [81]:
df_bottom[df_bottom.user.isin(user_list)].shape

(633, 147)

In [16]:
Counter(list(df.label))

Counter({0: 206, 1: 83, 2: 126, 3: 135, 4: 102})

In [17]:
#np.unique(df_bottom.label)

In [8]:
#np.unique(df_middle.label)

In [9]:
#np.unique(df_upper.label)

In [51]:
#val_type =  "person-out"
#val_type =  "10 fold"

In [90]:
#df.head()
target_dir = "/Users/y-nakamura/yugo_workspace/bar_exercise/data/clf_results/2021_dpsws"
os.chdir(target_dir)

In [91]:
df = df_upper[df_upper.user.isin(user_list)]

In [92]:
condition = "upper_PD_window128slide64_10-fold"
#condition = "upper_PD_window64slide32_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PD(df, target_dir, target_class, condition, classifiers, features_list)

RF
10 fold
1


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


1 RF fin
2


[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


2 RF fin
3


...

4 LightGBM fin
5
5 LightGBM fin
6


/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished
/Users/y-nakamur

6 LightGBM fin
7
7 LightGBM fin
8


[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.1s finished


8 LightGBM fin


In [93]:
condition = "upper_PI_window128slide64_LOPO"
#condition = "upper_PI_window64slide32_LOPO"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.5s finished


RF fin
ANN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.5s remaining:    0.5s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.9s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.1s remaining:    0.1s


ANN fin
LR


...

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


SVM fin
KNN
KNN fin
LightGBM


[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.7s remaining:    1.7s


LightGBM fin


[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    2.7s finished


In [84]:
# condition = "middle_window64slide32_10-fold"
# target_class = "bar_exercise"
# os.makedirs(condition, exist_ok=True)
# os.makedirs(condition +'/clf_reports', exist_ok=True)

# features_list = acc_feature145_names
# run_closs_val(df_middle, target_dir, target_class, condition, classifiers, features_list, val_type)

In [89]:
target_dir = "/Users/y-nakamura/yugo_workspace/bar_exercise/data/clf_results/2021_dpsws"
os.chdir(target_dir)

df = df_bottom[df_bottom.user.isin(user_list)]

condition = "bottom_PD_window128slide64_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PD(df, target_dir, target_class, condition, classifiers, features_list)

condition = "bottom_PI_window128slide64_LOPO"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF
10 fold
1


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


1 RF fin
2


[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


2 RF fin
3


...

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.1s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_clas

KNN fin
LightGBM


[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.6s remaining:    1.6s


LightGBM fin


[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    2.6s finished


In [88]:
target_dir = "/Users/y-nakamura/yugo_workspace/bar_exercise/data/clf_results/2021_dpsws"
os.chdir(target_dir)
df = df_middle[df_middle.user.isin(user_list)]

condition = "middle_PD_window128slide64_10-fold"
#condition = "upper_PD_window64slide32_10-fold"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PD(df, target_dir, target_class, condition, classifiers, features_list)

condition = "middle_PI_window128slide64_LOPO"
#condition = "upper_PI_window64slide32_LOPO"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok=True)
os.makedirs(condition +'/clf_reports', exist_ok=True)
features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF
10 fold
1


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


1 RF fin
2


[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    0.2s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


2 RF fin
3


...

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.1s finished
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/y-nakamura/miniforge3/envs/dev_env/lib/python3.8/site-packages/sklearn/metrics/_clas

SVM fin
KNN
KNN fin
LightGBM


[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.7s remaining:    1.7s


LightGBM fin


[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    2.7s finished


In [96]:
# condition = "upper_window64slide32_10-fold"
# target_class = "bar_exercise"
# os.makedirs(condition, exist_ok=True)
# os.makedirs(condition +'/clf_reports', exist_ok=True)

# features_list = acc_feature145_names
# run_closs_val(df_upper, target_dir, target_class, condition, classifiers, features_list, val_type)

# Feature Importance

# Majority vote

In [33]:
def sliding_window(data, window, step_size):
    shape = (int(data.shape[-1] / window * window / step_size - 1), window)
    strides = (data.strides[-1] * step_size, data.strides[-1])
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

def majority_filter(y_pred, window_size):
    sliding_y_pred  = sliding_window(y_pred, window_size, 1)
    voted_y_pred = []
    for y_list in sliding_y_pred:
        voted_y_pred.append(collections.Counter(y_list).most_common()[0][0])
    voted_y_pred.append(y_list[-1])
    
    return voted_y_pred

In [34]:
def run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num):
    print(in_condition, ",", out_condition)
    os.makedirs(out_condition, exist_ok=True)
    target_files = [tf for tf in os.listdir(target_dir + "/" + in_condition )  if tf.startswith( in_condition+ '_'+target_class +'_y_pred')]
    target_files.sort()
    print(target_files)
    print(target_files[0])
    os.makedirs(out_condition +'/clf_reports', exist_ok=True)
    for target_file in target_files:
        df = pd.read_csv(target_dir  + "/" + in_condition + "/"+ target_file, index_col=[0])
        model_name = target_file[:-4].split("_")[-1]
        #print(model_name)

        output_file = target_dir + "/"+out_condition + '/'+out_condition + '_'+ target_class + '_y_pred_true_user_' + model_name +'.csv'

            
        df.y_pred = majority_filter(df.y_pred.values, vote_num)

        df.to_csv(output_file)

        labels = np.unique(df.y)

        cr_dict = classification_report(df.y, df.y_pred, output_dict=True)

        cr_df = pd.DataFrame(cr_dict).T

        cr_df.to_csv( target_dir + "/"+out_condition+ '/clf_reports' + '/'+out_condition + '_'+ target_class+ '_result_report_'+model_name+'.csv')

        cm = confusion_matrix(df.y, df.y_pred, labels=labels)
        cm_df = pd.DataFrame(cm, index=labels, columns=labels)
        cm_file_name =  target_dir + "/"+out_condition+ '/'+out_condition + '_'+ target_class + '_result_cm_' + model_name + ".png"
        generate_cm(cm_df, index=labels, columns=labels, filename = cm_file_name)


In [57]:
condition = "upper_window64slide32_LOPO"
target_class = "bar_exercise"
os.makedirs(condition, exist_ok= True)
os.makedirs(condition +'/clf_reports', exist_ok=True)

features_list = acc_feature145_names
run_closs_val_PI(df, target_dir, target_class, condition, classifiers, features_list)

RF


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.1s finished


RF fin
ANN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    1.1s remaining:    1.1s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    1.7s finished


ANN fin
LR


...

SVM fin
KNN


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    0.0s finished


KNN fin
LightGBM


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   4 out of   8 | elapsed:    3.4s remaining:    3.4s
[Parallel(n_jobs=7)]: Done   8 out of   8 | elapsed:    5.3s finished


LightGBM fin


In [65]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 10
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)


upper_window64slide32_LOPO , upper_window64slide32_LOPO_majority10
['upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_DT.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ET.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_KNN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LR.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LightGBM.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_NB.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_RF.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_SVM.csv']
upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv


In [66]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 5
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)


upper_window64slide32_LOPO , upper_window64slide32_LOPO_majority5
['upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_DT.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ET.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_KNN.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LR.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_LightGBM.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_NB.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_RF.csv', 'upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_SVM.csv']
upper_window64slide32_LOPO_bar_exercise_y_pred_true_user_ANN.csv


In [ ]:
in_condition = "upper_window64slide32_LOPO"
vote_num = 3
out_condition = "upper_window64slide32_LOPO_majority" + str(vote_num)

target_class = "bar_exercise"
run_majority_vote(target_dir, target_class, in_condition, out_condition, vote_num)
